## Thread RLock with mutiple threads.

In this section we will see that how an RLock is usefull when multiple thread try to access it.

we will aslo see that how a RLock is acquired in a nested way and in order to acquire it by another thread we need to relase it in the nested way. 

### Example: 1 RLock with mutiple thread.


**----->> Acquiring lock two times**

**----->> No releasing of lock**

In this section we will see a simple mechanism to acquire a RLock object by the multiple threads and we will see that which thread is able to acquire the lock how and many time it acquire the lock.

`Note:` Here we should note that we are acquiring the lock two time but we wont release the lock at all it means only one thread will be able to acquire the both locks, and allow to use the defined piece of code.

first we will define our module and logging basicConfig.

In [1]:
import threading
import time
import logging
from random import randint

In [2]:
# setting logging config
logging.basicConfig(level=logging.DEBUG,
                   format='%(threadName)s::%(funcName)s: %(message)s')

# a function for logs
def log(msg):
    logging.debug(msg)

Now we will create two function which will use the RLock() object to be acquired.

In [25]:
# func1 

def func1(lock):
    logging.debug("Started!")
    logging.debug("Trying Acquiring lock!")
    time.sleep(1)
    if lock.acquire(0):  
        # here we need to set the blocking to resume the code execution.
        logging.debug("Lock Acquired!<<--------Success-------->>")
    else:
        logging.debug("uable to acquire the lock!<<--------Failed-------->>")
        
    # now in every call we will sleep for 5 seconds.
    time.sleep(5)
    
    
def func2(lock):
    
    logging.debug("Started!")
    logging.debug("Trying Acquiring lock!")
    time.sleep(1)
    if lock.acquire(0):  
        # here we need to set the blocking to resume the code execution.
        logging.debug("Lock Acquired!<<--------Success-------->>")
    else:
        logging.debug("uable to acquire the lock!<<--------Failed-------->>")
        
    # now in every call we will sleep for 3 seconds.
    time.sleep(3)

Now both the function are sharing the same lock it means a therad which acqure the func1 lock it will also able to acquire the lock from func2.

Now we need to make a subclass so we can run both the function by same lock with more control.

In [28]:
class MyThread(threading.Thread):
    
    def __init__(self, lock=None, name=None, delay=None):
        
        threading.Thread.__init__(self, name=name)
        
        self.name = name
        self.lock = lock
        self.delay = delay
     
    # Now we will override the run method.
    def run(self):
        
        logging.debug('Starting!')
        # now we will acquire the lock and we will set the random time
        # for acquiring the lock we can see the real effect
        time.sleep(randint(1,10))  
        
        # now we will start the func1,
        func1(self.lock)
        logging.debug("Wating to access the fuc2!")
        time.sleep(self.delay)
        func2(self.lock)
        
        logging.debug("Exiting")
        

`Explanation:` Now we will understand that what will happen when a MyThread instance will start.

1. first of all it will show the starting message.

2. since we are sleeping for random time so we won't know that which thread will access the func1.

3. Now suppose that threre are three threads ("Thread1", "Thread2", "Thread3") and "Thread2" is able to acess the func1 first.

4. if "Thread2" access the func1 first then it will acquire the lock first.

5. Now we will give the delay time more than 5sec so make some randomness of acquiring the func2 lock.

6. Now for each thread which tried to access the lock it will show the message according to the Success or Faliur.

7. Finally we will Exiting from each thread.

In [29]:
# first of all we need to create a RLock object

rlock = threading.RLock()

# Creating three theadss
for i in range(1,4):
    rt = randint(5,10)
    # given the random value for waiting time in secodns
    # to access the func2
    thread = MyThread(name=f"Thread-{i}", lock=rlock, delay=rt)
    thread.start()

Thread-1::run: Starting!
Thread-2::run: Starting!
Thread-3::run: Starting!
Thread-2::func1: Started!
Thread-2::func1: Trying Acquiring lock!
Thread-2::func1: Lock Acquired!<<--------Success-------->>
Thread-3::func1: Started!
Thread-3::func1: Trying Acquiring lock!
Thread-2::run: Wating to access the fuc2!
Thread-1::func1: Started!
Thread-1::func1: Trying Acquiring lock!
Thread-3::func1: uable to acquire the lock!<<--------Failed-------->>
Thread-1::func1: uable to acquire the lock!<<--------Failed-------->>
Thread-3::run: Wating to access the fuc2!
Thread-1::run: Wating to access the fuc2!
Thread-2::func2: Started!
Thread-2::func2: Trying Acquiring lock!
Thread-2::func2: Lock Acquired!<<--------Success-------->>
Thread-3::func2: Started!
Thread-3::func2: Trying Acquiring lock!
Thread-2::run: Exiting
Thread-3::func2: uable to acquire the lock!<<--------Failed-------->>
Thread-1::func2: Started!
Thread-1::func2: Trying Acquiring lock!
Thread-3::run: Exiting
Thread-1::func2: uable to acq

In [33]:
from IPython.display import Image

Now we will try to understand that what just happened.

In [34]:
Image(url='char1.jpeg')

`Conclusion:` From the above chart you can see that how thread2 get the first access to the func1 and acquire the lock and since the lock is already acquired by the thread2 then thread1 and thread3 won't be able to acquire the lock.

* As all threads exit from func1 they will wait for the accessing the func2.

* Now each thread will try to acquire the again, but thread1 didn't release the first acquire of the lock so again thread1 and thread2 will not be able to acquire the lock.

* In this process thread1 acquire the lock two time but do not release the lock.

`Note:` A thread acquire the lock in a nested way so it has to be release the lock in nested way.

___
___

### Example: 2 RLock with mutiple thread.


**----->> Acquiring lock two times**

**----->> releasing lock one time  (part 1)**

In this section we will release the lock from the func2 after exiting from func2.

* Since we have a time delay (random time) after acquiring the the thread so there may a chance where a thread will try to acquire the 
    func2 lock after releasing lock by previous thread.

In [5]:

import threading
import time
import logging
from random import randint

# ------------------------------------------------------------------- #

# setting logging config
logging.basicConfig(level=logging.DEBUG,
                   format='%(threadName)s::%(funcName)s: %(message)s')

# ------------------------------------------------------------------- #


# func1 

def func1(lock):
    logging.debug("Started!")
    logging.debug("Trying Acquiring lock!")
    time.sleep(1)
    if lock.acquire(0):  
        # here we need to set the blocking to resume the code execution.
        logging.debug("Lock Acquired!<<--------Success-------->>")
    else:
        logging.debug("uable to acquire the lock!<<--------Failed-------->>")
        
    # now in every call we will sleep for 5 seconds.
    time.sleep(5)
    
    
def func2(lock):
    
    logging.debug("Started!")
    logging.debug("Trying Acquiring lock!")
    rt1 = randint(1,5)
    rt2 = randint(1,5)
    time.sleep(rt1)
    if lock.acquire(0):  
        # here we need to set the blocking to resume the code execution.
        logging.debug("Lock Acquired!<<--------Success-------->>")
         # now in every call we will sleep for 3 seconds
        time.sleep(rt2)
        
        logging.debug("Releasing lock!")
        lock.release()
        logging.debug("Lock Released!..")
        
    else:
        logging.debug("uable to acquire the lock!<<--------Failed-------->>")
         # now in every call we will sleep for 3 seconds
        time.sleep(rt2)
    
    
    # here we will release a lock if it is acquired.
        
        
    
# ---------------------------------------------------------------------------------------------------------- #
# ---------------------------------------------------------------------------------------------------------- #

class MyThread(threading.Thread):
    
    def __init__(self, lock=None, name=None, delay=None):
        
        threading.Thread.__init__(self, name=name)
        
        self.name = name
        self.lock = lock
        self.delay = delay
     
    # Now we will override the run method.
    def run(self):
        
        logging.debug('Starting!')
        # now we will acquire the lock and we will set the random time
        # for acquiring the lock we can see the real effect
        time.sleep(randint(1,10))  
        
        # now we will start the func1,
        func1(self.lock)
        logging.debug("Wating to access the fuc2!")
        time.sleep(self.delay)
        func2(self.lock)
        
        logging.debug("Exiting")
        
# ---------------------------------------------------------------------------------------------------------- #
# ---------------------------------------------------------------------------------------------------------- #

rlock = threading.RLock()

# Creating three theadss
for i in range(1,4):
    rt = randint(5,10)
    # given the random value for waiting time in secodns
    # to access the func2
    thread = MyThread(name=f"Thread-{i}", lock=rlock, delay=rt)
    thread.start()

Thread-1::run: Starting!
Thread-2::run: Starting!
Thread-3::run: Starting!
Thread-1::func1: Started!
Thread-1::func1: Trying Acquiring lock!
Thread-1::func1: Lock Acquired!<<--------Success-------->>
Thread-3::func1: Started!
Thread-2::func1: Started!
Thread-3::func1: Trying Acquiring lock!
Thread-2::func1: Trying Acquiring lock!
Thread-2::func1: uable to acquire the lock!<<--------Failed-------->>
Thread-3::func1: uable to acquire the lock!<<--------Failed-------->>
Thread-1::run: Wating to access the fuc2!
Thread-3::run: Wating to access the fuc2!
Thread-2::run: Wating to access the fuc2!
Thread-1::func2: Started!
Thread-1::func2: Trying Acquiring lock!
Thread-2::func2: Started!
Thread-2::func2: Trying Acquiring lock!
Thread-1::func2: Lock Acquired!<<--------Success-------->>
Thread-3::func2: Started!
Thread-3::func2: Trying Acquiring lock!
Thread-2::func2: uable to acquire the lock!<<--------Failed-------->>
Thread-2::run: Exiting
Thread-1::func2: Releasing lock!
Thread-1::func2: Lo

`Note:` Note here after releasing the lock of func2 by thread1, thread3 try to acquire the lock but it is unable to acquire the lock.

* To acquire a RLock object by a thread, it must be fully unlocked, means it should unlocked all the nestead level applied by the another thread which acquired it multiple times.

* In another word when a thread acquires a lock its acquiring level increases, for example if a thread1 acquire RLock object by 4-time the it's acquiring level will be 4, so it must be released by 4 time make it fully unlocked, so other thread can acquire it

____
____


### Example: 3 RLock with mutiple thread.


**----->> Acquiring lock two times**

**----->> releasing lock one time  (part 2 with acqiring timeout)**

In this section we will release the lock from the func2 after exiting from func2.

* Since we have a time delay (random time) after acquiring the the thread so there may a chance where a thread will try to acquire the 
    func2 lock after releasing lock by previous thread.
    
* `Note:` Now we will add the random timeout for other thread who are trying to acquiring the lock.

In [9]:

import threading
import time
import logging
from random import randint

# ------------------------------------------------------------------- #

# setting logging config
logging.basicConfig(level=logging.DEBUG,
                   format='%(threadName)s::%(funcName)s: %(message)s')

# ------------------------------------------------------------------- #


# func1 

def func1(lock):
    logging.debug("Started!")
    logging.debug("Trying Acquiring lock!")
    time.sleep(1)
    if lock.acquire(0):  
        # here we need to set the blocking to resume the code execution.
        logging.debug("Lock Acquired!<<--------Success-------->>")
    else:
        logging.debug("uable to acquire the lock!<<--------Failed-------->>")
        
    # now in every call we will sleep for 5 seconds.
    time.sleep(5)
    
    
def func2(lock):
    
    logging.debug("Started!")
    logging.debug("Trying Acquiring lock!")
    rt1 = randint(1,5)
    rt2 = randint(1,5)
    rt3 = randint(3,6)
    time.sleep(rt1)
    if lock.acquire(timeout=rt3):  
        # here we need to set the blocking to resume the code execution.
        logging.debug("Lock Acquired!<<--------Success-------->>")
         # now in every call we will sleep for 3 seconds
        time.sleep(rt2)
        
        logging.debug("Releasing lock!")
        lock.release()   # <<----------------------------------------:
        logging.debug("Lock Released!..")
        
    else:
        logging.debug("uable to acquire the lock!<<--------Failed-------->>")
         # now in every call we will sleep for 3 seconds
        time.sleep(rt2)
    
    
    # here we will release a lock if it is acquired.
        
        
    
# ---------------------------------------------------------------------------------------------------------- #
# ---------------------------------------------------------------------------------------------------------- #

class MyThread(threading.Thread):
    
    def __init__(self, lock=None, name=None, delay=None):
        
        threading.Thread.__init__(self, name=name)
        
        self.name = name
        self.lock = lock
        self.delay = delay
     
    # Now we will override the run method.
    def run(self):
        
        logging.debug('Starting!')
        # now we will acquire the lock and we will set the random time
        # for acquiring the lock we can see the real effect
        time.sleep(randint(1,10))  
        
        # now we will start the func1,
        func1(self.lock)
        logging.debug("Wating to access the fuc2!")
        time.sleep(self.delay)
        func2(self.lock)
        
        logging.debug("Exiting")
        
# ---------------------------------------------------------------------------------------------------------- #
# ---------------------------------------------------------------------------------------------------------- #

rlock = threading.RLock()

# Creating three theadss
for i in range(1,4):
    rt = randint(5,10)
    # given the random value for waiting time in secodns
    # to access the func2
    thread = MyThread(name=f"Thread-{i}", lock=rlock, delay=rt)
    thread.start()

Thread-1::run: Starting!
Thread-2::run: Starting!
Thread-3::run: Starting!
Thread-3::func1: Started!
Thread-3::func1: Trying Acquiring lock!
Thread-3::func1: Lock Acquired!<<--------Success-------->>
Thread-2::func1: Started!
Thread-1::func1: Started!
Thread-2::func1: Trying Acquiring lock!
Thread-1::func1: Trying Acquiring lock!
Thread-1::func1: uable to acquire the lock!<<--------Failed-------->>
Thread-2::func1: uable to acquire the lock!<<--------Failed-------->>
Thread-3::run: Wating to access the fuc2!
Thread-1::run: Wating to access the fuc2!
Thread-2::run: Wating to access the fuc2!
Thread-3::func2: Started!
Thread-3::func2: Trying Acquiring lock!
Thread-2::func2: Started!
Thread-2::func2: Trying Acquiring lock!
Thread-3::func2: Lock Acquired!<<--------Success-------->>
Thread-1::func2: Started!
Thread-1::func2: Trying Acquiring lock!
Thread-3::func2: Releasing lock!
Thread-3::func2: Lock Released!..
Thread-3::run: Exiting
Thread-2::func2: uable to acquire the lock!<<--------Fa

`Note:` Here also the conclusion is simple that Thread-3 has acquired both the function lock but it release the func2 lock before 'Thread-1' and 'Thread-2' try to acquire it.

* As we know that Thread-1 and Thread-2 can't acquire the lock since it is not fully unlocked.

`*Note*:` Try to simulate multiple time until lock is released before trying to acquire the lock by both remaining threads.

___
___


### Example: 4 RLock with mutiple thread.


**----->> Acquiring lock two times**

**----->> releasing the lock completely from func2**

Now in this section we will release the lock completely from func2, means we will release the lock two times.

* Since we have a time delay (random time) after acquiring the the thread so there may a chance where a thread will try to acquire the 
    func2 lock after releasing lock by previous thread.
    
* `Note:` Now we will add the random timeout for other thread who are trying to acquiring the lock. it will allow us to simulate real kind of problems.

In [14]:

import threading
import time
import logging
from random import randint

# ------------------------------------------------------------------- #

# setting logging config
logging.basicConfig(level=logging.DEBUG,
                   format='%(threadName)s::%(funcName)s: %(message)s')

# ------------------------------------------------------------------- #


# func1 

def func1(lock):
    logging.debug("Started!")
    logging.debug("Trying Acquiring lock!")
    time.sleep(1)
    if lock.acquire(0):  
        # here we need to set the blocking to resume the code execution.
        logging.debug("Lock Acquired!<<--------Success-------->>")
    else:
        logging.debug("uable to acquire the lock!<<--------Failed-------->>")
        
    # now in every call we will sleep for 5 seconds.
    time.sleep(5)
    
    
def func2(lock):
    
    logging.debug("Started!")
    logging.debug("Trying Acquiring lock!")
    rt1 = randint(1,5)
    rt2 = randint(1,5)
    rt3 = randint(3,6)
    time.sleep(rt1)
    if lock.acquire(timeout=rt3):  
        # here we need to set the blocking to resume the code execution.
        logging.debug("Lock Acquired!<<--------Success-------->>")
         # now in every call we will sleep for 3 seconds
        time.sleep(rt2)
        
        logging.debug("Releasing lock!")
        # Now here we need to release the lock two times  <<----------------------------------------:
        lock.release()   # <<----------------------------------------:
        logging.debug("Lock Released!..")
        
        # Note: Here since at this point two or more than two threads will try to  release lock
        # But in our code (at this stage) there is very low chance that after releasing the lock two time from 
        # very first thread, they will acquire two times so we will use try and except statement.
        
        try:
            logging.debug("Releasing lock!")
            lock.release()   # <<----------------------------------------:
            logging.debug("Lock Released!..")
        except RuntimeError:
            logging.debug("<<<..Don't owns any lock.!..>>>")
        except:
            logging.debug("<<Lock is already fully released!..>>")
        
        
    else:
        logging.debug("uable to acquire the lock!<<--------Failed-------->>")
         # now in every call we will sleep for 3 seconds
        time.sleep(rt2)
    
    
    # here we will release a lock if it is acquired.
        
        
    
# ---------------------------------------------------------------------------------------------------------- #
# ---------------------------------------------------------------------------------------------------------- #

class MyThread(threading.Thread):
    
    def __init__(self, lock=None, name=None, delay=None):
        
        threading.Thread.__init__(self, name=name)
        
        self.name = name
        self.lock = lock
        self.delay = delay
     
    # Now we will override the run method.
    def run(self):
        
        logging.debug('Starting!')
        # now we will acquire the lock and we will set the random time
        # for acquiring the lock we can see the real effect
        time.sleep(randint(1,10))  
        
        # now we will start the func1,
        func1(self.lock)
        logging.debug("Wating to access the fuc2!")
        time.sleep(self.delay)
        func2(self.lock)
        
        logging.debug("Exiting")
        
# ---------------------------------------------------------------------------------------------------------- #
# ---------------------------------------------------------------------------------------------------------- #

rlock = threading.RLock()

# Creating three theadss
for i in range(1,4):
    rt = randint(5,10)
    # given the random value for waiting time in secodns
    # to access the func2
    thread = MyThread(name=f"Thread-{i}", lock=rlock, delay=rt)
    thread.start()

Thread-1::run: Starting!
Thread-2::run: Starting!
Thread-3::run: Starting!
Thread-3::func1: Started!
Thread-1::func1: Started!
Thread-3::func1: Trying Acquiring lock!
Thread-1::func1: Trying Acquiring lock!
Thread-3::func1: Lock Acquired!<<--------Success-------->>
Thread-1::func1: uable to acquire the lock!<<--------Failed-------->>
Thread-2::func1: Started!
Thread-2::func1: Trying Acquiring lock!
Thread-2::func1: uable to acquire the lock!<<--------Failed-------->>
Thread-3::run: Wating to access the fuc2!
Thread-1::run: Wating to access the fuc2!
Thread-2::run: Wating to access the fuc2!
Thread-1::func2: Started!
Thread-3::func2: Started!
Thread-1::func2: Trying Acquiring lock!
Thread-3::func2: Trying Acquiring lock!
Thread-2::func2: Started!
Thread-2::func2: Trying Acquiring lock!
Thread-3::func2: Lock Acquired!<<--------Success-------->>
Thread-3::func2: Releasing lock!
Thread-3::func2: Lock Released!..
Thread-3::func2: Releasing lock!
Thread-3::func2: Lock Released!..
Thread-2::f

`Conclusion:` Here you can see that another real life example of the acquiring and releasing the locks, no we will try to understand the above simulation and we will focus on the acquiring and releasing.

1. Thread-3 acquires the func1 lock, Thread-1, thraea-2 are unable to acquire.

2. all threads are waiting for to access the fucn2 

3. Since Thread-3 has already acquired the func1 lock and did't release it yet so, it will acquire the func2 lock also.

4. Thread-3:: immediately acquiring the lock it release the lock two time, now lock is completely unlocked and ready to be acquired by another thread. 

5. Now Thread-2 Acquires the func2 lock it's waiting timeout was large enoungh to acquire the lock. (Thread-3 is dead by now.)

6. Now thread-2: release it lock only one time because it's acquiring level was only 1, and releasing of second time it show it don't own any lock. 

7. Now Thread-1 waiting timeout was also large enough to acquire the func2 lock also, so it acquire it, (by this time Thread-2 is dead.)

8. Now Thread-1 release the lock one time and second time it say's (don't own any lock) since it's acquiring level was only 1.

9. Thread-1 also exits.

___
___


### Example: 5 RLock with mutiple thread.


**----->> Acquiring lock two times**

**----->> releasing locks after a dealy**

**------>> More Real world like simulation**

Now in this section we will release the lock completely after acquiring the lock, func1 lock will be released in the fun1 and func2 lock will be released in func2 lock. (This will be more real life problem based locking.)

* Since both locks will be released in their own func, there will be enough time to acquire the lock by each thread in both func.
    
* `Note:` Now we will add the random timeout for other thread who are trying to acquiring the lock. it will allow us to simulate real kind of problems.

In [16]:

import threading
import time
import logging
from random import randint

# ------------------------------------------------------------------- #

# setting logging config
logging.basicConfig(level=logging.DEBUG,
                   format='%(threadName)s::%(funcName)s: %(message)s')

# ------------------------------------------------------------------- #


# func1 

def func1(lock):
    logging.debug("Started!")
    logging.debug("Trying Acquiring lock!")
    
    rt1 = randint(1,5)
    rt2 = randint(1,5)
    rt3 = randint(3,6)
    time.sleep(rt1)
    
    if lock.acquire(timeout=rt3):  
        # here we need to set the blocking to resume the code execution.
        logging.debug("Lock Acquired!<<--------Success-------->>")
        time.sleep(rt2)
        try:
            logging.debug("Releasing lock!")
            lock.release()   # <<----------------------------------------:
            logging.debug("Lock Released!..")
        except RuntimeError:
            logging.debug("<<<..Don't owns any lock.!..>>>")
            
    else:
        time.sleep(rt2)
        logging.debug("uable to acquire the lock!<<--------Failed-------->>")
        
    # now in every call we will sleep for 5 seconds.
   
    

    
def func2(lock):
    
    logging.debug("Started!")
    logging.debug("Trying Acquiring lock!")
    rt1 = randint(1,5)
    rt2 = randint(1,5)
    rt3 = randint(3,6)
    
    time.sleep(rt1)
    if lock.acquire(timeout=rt3):  
        # here we need to set the blocking to resume the code execution.
        logging.debug("Lock Acquired!<<--------Success-------->>")
         # now in every call we will sleep for 3 seconds
        time.sleep(rt2)
        
        # Note: Here since at this point two or more than two threads will try to  release lock    
        try:
            logging.debug("Releasing lock!")
            lock.release()   # <<----------------------------------------:
            logging.debug("Lock Released!..")
        except RuntimeError:
            logging.debug("<<<..Don't owns any lock.!..>>>")
        
        
        
    else:
        logging.debug("uable to acquire the lock!<<--------Failed-------->>")
         # now in every call we will sleep for 3 seconds
        time.sleep(rt2)
    
    
    # here we will release a lock if it is acquired.
        
        
    
# ---------------------------------------------------------------------------------------------------------- #
# ---------------------------------------------------------------------------------------------------------- #

class MyThread(threading.Thread):
    
    def __init__(self, lock=None, name=None, delay=None):
        
        threading.Thread.__init__(self, name=name)
        
        self.name = name
        self.lock = lock
        self.delay = delay
     
    # Now we will override the run method.
    def run(self):
        
        logging.debug('Starting!')
        # now we will acquire the lock and we will set the random time
        # for acquiring the lock we can see the real effect
        time.sleep(randint(1,10))  
        
        # now we will start the func1,
        func1(self.lock)
        logging.debug("Wating to access the fuc2!")
        time.sleep(self.delay)
        func2(self.lock)
        
        logging.debug("Exiting")
        
# ---------------------------------------------------------------------------------------------------------- #
# ---------------------------------------------------------------------------------------------------------- #

rlock = threading.RLock()

# Creating three theadss
for i in range(1,4):
    rt = randint(5,10)
    # given the random value for waiting time in secodns
    # to access the func2
    thread = MyThread(name=f"Thread-{i}", lock=rlock, delay=rt)
    thread.start()

Thread-1::run: Starting!
Thread-2::run: Starting!
Thread-3::run: Starting!
Thread-1::func1: Started!
Thread-1::func1: Trying Acquiring lock!
Thread-1::func1: Lock Acquired!<<--------Success-------->>
Thread-2::func1: Started!
Thread-2::func1: Trying Acquiring lock!
Thread-1::func1: Releasing lock!
Thread-1::func1: Lock Released!..
Thread-1::run: Wating to access the fuc2!
Thread-2::func1: Lock Acquired!<<--------Success-------->>
Thread-3::func1: Started!
Thread-3::func1: Trying Acquiring lock!
Thread-2::func1: Releasing lock!
Thread-2::func1: Lock Released!..
Thread-2::run: Wating to access the fuc2!
Thread-3::func1: Lock Acquired!<<--------Success-------->>
Thread-3::func1: Releasing lock!
Thread-3::func1: Lock Released!..
Thread-3::run: Wating to access the fuc2!
Thread-1::func2: Started!
Thread-1::func2: Trying Acquiring lock!
Thread-2::func2: Started!
Thread-2::func2: Trying Acquiring lock!
Thread-1::func2: Lock Acquired!<<--------Success-------->>
Thread-1::func2: Releasing lock!

`Conclusion:` By using the random timeout we have a simulation where all three thread are able to acquire the lock on both func1, this kind of mechanism can be applied on real life project by adding more functionality to the Thread subclass and RLock class.

`Note:` if we want to repeate func with mulitple value then we can use the RLock with condition object and and event also.

--------------------- ***THE END***--------------------- 